In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import cmath
from numba import njit, f8,i8,c16
import numba
import matplotlib.animation as animation
from IPython.display import HTML
import copy
import matplotlib
import random
import  concurrent.futures
import func_for_paralell

import sys
sys.path.append('../../lb')
import RungeKutta_acceralated

In [ ]:
#パラメーター
nu = 0.001

f = (1 + 1j) * 5 * 0.001

Dimention = 14 #uの次元数
q = 2
k_0 = 2 ** (-4)
k_n = np.zeros((Dimention+4,1))
k_n[2:-2,0] = [k_0 * q ** (i+1) for i in range(Dimention)]
#print(k_n)

c_n_1 = copy.deepcopy(k_n)[2:-2, 0:]
c_n_1[-2:, 0:] = np.zeros((2,1))
#print(c_n_1)

beta = 0.5
c_n_2 = copy.deepcopy(k_n)[1:-3, 0:] * -beta
c_n_2[-1, 0] = np.zeros((1, 1))
#print(c_n_2)

c_n_3 = copy.deepcopy(k_n)[:-4, 0:] * (beta - 1)
#print(c_n_3)

#parameter = [nu,f,k_n,c_n_1,c_n_2,c_n_3]
parameter = {
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}
#print(parameter)
#print(k_n)

runge_para = {
    'start' :  np.load("14次元初期値.npy"),
    'step' : 0.00025,
    'period' : np.array([0,5000]),
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3,
    #'check' : 2000,
    #'progress' : 10
}

# コンテスト

In [ ]:
#c4が極大値を取る時の全変数(+時間)をndarrayで返す関数
@njit(f8[:,:](c16[:,:],f8[:,:]),cache=True)
def loc_max_7(arr,z_list):
    for j in range(len(arr[0,:]) - 3):
        #7点とって極大判定
        if (np.abs(arr[3,j+1]) - np.abs(arr[3,j]) > 0
        and np.abs(arr[3,j+2]) - np.abs(arr[3,j+1]) > 0
        and np.abs(arr[3,j+3]) - np.abs(arr[3,j+2]) > 0
        and np.abs(arr[3,j+4]) - np.abs(arr[3,j+3]) < 0
        and np.abs(arr[3,j+5]) - np.abs(arr[3,j+4]) < 0
        and np.abs(arr[3,j+6]) - np.abs(arr[3,j+5]) < 0):
            z_list = np.append(z_list,np.abs(arr[:,j+3:j+4]),axis=1)
    return z_list

In [ ]:
%%time
runge_para["period"]=(0,../laminar_initials/beta0.4215000000000002_nu0.00017072648529039704_step0.01_1000chec_10pro_10period.npz0
competitives = 10

plt.figure(figsize=(60,5))

# #nu
# runge_para["nu"] = 0.0001697
# #betaの値に従いc_n_2,c_n_3を更新
# beta = 0.42119

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#摂動を与えた初期値のリスト
starts = np.zeros((14,competitives))
starts = starts.astype(np.complex128)

lmodel = Long_laminar() #インスタンス化
for i in range(competitives):
    starts[:,i:i+1] = lmodel.perturbator(runge_para['start'])

#ラミナーから出た極大値の時間
times = np.zeros((competitives,1))

for i in tqdm(range(competitives)):
    cut_arr = np.zeros((Dimention+1,1))
    
    runge_para['start'] = starts[:,i:i+1]
    #インスタンス化&実行
    model = RungeKutta_acceralated.RungeKutta()
    model.get_arr_latter(nu = runge_para['nu'],f=runge_para['f'],k_n=runge_para['k_n'],c_n_1=runge_para['c_n_1'],c_n_2=runge_para['c_n_2'],c_n_3=runge_para['c_n_3'],start=runge_para['start'], step=runge_para['step'],period=runge_para['period'],latter=1)
    #プロット
    plt.subplot(1,1,1)
    plt.ylabel(f"u{4}")
    plt.yscale=("log")
    plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[0,::skip]),lw=0.5)
    
    #極大抽出
    cut_arr = loc_max_7(model.arr_latter,cut_arr)
    
    for j in range(len(cut_arr[0,:])):
        if not ((0.52 < cut_arr[0,j] < 0.68 or 0.78 < cut_arr[0,j] < 1) 
            and (0.02 < cut_arr[1,j] < 0.13 or 0.19 < cut_arr[1,j] < 0.34)
            and (0.12 < cut_arr[2,j] < 0.26 or 0.36 < cut_arr[2,j] < 0.48)
            and (0.17 < cut_arr[3,j] < 0.24 or 0.25 < cut_arr[3,j] < 0.32 or 0.34 < cut_arr[3,j] < 0.42)
            and (0.08 < cut_arr[4,j] < 0.2 or 0.28 < cut_arr[4,j] < 0.35)
            and (0.05 < cut_arr[5,j] < 0.115 or 0.12 < cut_arr[5,j] < 0.155)
            and (0.03 < cut_arr[6,j] < 0.07 or 0.1 < cut_arr[6,j] < 0.22)
            and (0.02 < cut_arr[7,j] < 0.08 or 0.104 < cut_arr[7,j] <0.144)
            and (0.014 < cut_arr[8,j] < 0.078)
            and (0 < cut_arr[9,j] < 0.036 or 0.048 < cut_arr[9,j] < 0.076 or 0.084 < cut_arr[9,j] < 0.1)
            and (0 < cut_arr[10,j] < 0.014 or 0.02 < cut_arr[10,j] < 0.038 or 0.046 < cut_arr[10,j] < 0.068)
            and (0 < cut_arr[11,j] < 0.002 or 0.003 < cut_arr[11,j] < 0.0095 or 0.0135 < cut_arr[11,j] < 0.0225)
            and (0 < cut_arr[12,j] < 0.0006 or 0.001 < cut_arr[12,j] < 0.0024)
            and (0 < cut_arr[13,j] < 0.000005 or 0.00001 < cut_arr[13,j] < 0.000045)):
            times[i,0] = cut_arr[-1,j]
            break
            
#plt.savefig(fname=f"../laminar/contest_a=-8_b=-15_nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_{Dimention}dim.png")

print(np.mean(times))
print(np.std(times))
print(np.min(times))
print(np.max(times))

In [ ]:
#ある程度持続すれば更新
runge_para["period"]=np.array([0,1000])
competitives = 1000

#刻み幅
runge_para["step"] = 0.01
skip= 100

#runge_para['start'] = np.load("14次元初期値.npy")

plus = 1E-5

#nu
runge_para["nu"] =  0.000170863309
original_nu = runge_para["nu"]*1
#betaの値に従いc_n_2,c_n_3を更新
beta = 0.42119

lmodel = Long_laminar()

#摂動を与えた初期値のリスト作成
starts = np.zeros((14,competitives),dtype = np.complex128)
for i in range(competitives):
    starts[:,i:i+1] = perturbator(runge_para['start'])
    

#ラミナーから外れた時間
times = np.zeros((competitives,1))

for i in tqdm(range(competitives)):
    cut_arr = np.zeros((Dimention+1,1))
    
    runge_para['start'] = starts[:,i:i+1]
    #インスタンス化&実行
    lmodel.get_arr_latter(nu = runge_para['nu'],f=runge_para['f'],k_n=runge_para['k_n'],c_n_1=runge_para['c_n_1'],c_n_2=runge_para['c_n_2'],c_n_3=runge_para['c_n_3'],start=runge_para['start'], step=runge_para['step'],period=runge_para['period'],latter=1)
    
    #極大抽出
    loc_max = lmodel.loc_max_7(lmodel.arr) #u1が極大値なものを拾ってくる
    loc_min = lmodel.loc_min_7(lmodel.arr) #u1が極小値なものを拾ってくる
    
    #極大値での
    if (0.5 < np.min(np.abs(lmodel.arr[0,:])) and np.max(np.abs(lmodel.arr[0,:])) < 1.1 and 0.87 < np.min(loc_max[0,:]) and np.max(loc_min[0,:]) < 0.64):
        break

# ラミナーに留まる(パラメータ更新)

# 初期値・nu ランダム

In [ ]:
#ある程度持続すれば更新
runge_para["period"]=np.array([0,1000])
competitives = 800

#刻み幅
runge_para["step"] = 0.01
skip= 100

runge_para['start'] = np.load("../laminar_initials/beta0.423_nu0.00017584784643038092_step0.01_1500check_500pro_20000period.npz")["start"]

plus = - 1E-4

#nu
runge_para["nu"] = 0.00014193346213826156
#betaの値に従いc_n_2,c_n_3を更新
beta = 0.4327

model = RungeKutta_acceralated.RungeKutta()
lmodel = Long_laminar()

while True:
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
    for _ in range(3):
        #摂動を与えたnuのリスト作成
        nus = np.zeros((competitives),dtype = np.float64)
        nus[0] = runge_para["nu"]*1
        for i in range(competitives-1):
            a=-5
            b=-9
            nus[i+1] = runge_para["nu"] + (10) ** ((b - a) * np.random.rand() + a) * (-1) ** np.random.randint(1,3) 

        #摂動を与えた初期値のリスト作成
        starts = np.zeros((14,competitives),dtype = np.complex128)
        starts[:,0:1] = runge_para["start"].copy()
        for i in range(competitives-1):
            starts[:,i+1:i+2] = lmodel.perturbator(runge_para['start'])

        #ラミナーから外れた時間
        times = np.zeros((competitives)) + runge_para["period"][1]

        for i in range(competitives):
            runge_para['start'] = starts[:,i:i+1].copy()
            runge_para['nu'] = nus[i]
            #インスタンス化&実行
            model.get_arr_latter(**runge_para,latter=1)

            #極大抽出
            loc_max = Long_laminar.loc_max_7(model.arr_latter) #u1が極大値なものを拾ってくる
            loc_min = Long_laminar.loc_min_7(model.arr_latter) #u1が極小値なものを拾ってくる

            #極大値で判定
            for j in range(loc_max.shape[1]):
                if not 0.87 < loc_max[0,j] < 1.1:
                    times[i] = loc_max[-1, j]
                    break
            #極小値で判定
            for j in range(loc_min.shape[1]):
                if not 0.5 < loc_min[0,j] < 0.64:
                    times[i] = min([loc_min[-1,j], times[i]])
        runge_para['nu'] = nus[np.argmax(times)]
        runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
        print(max(times))
        if max(times) > runge_para["period"][1]*0.99:
            break

    if max(times) > 800:
        np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", runge_para['start'])
    print(f"beta={beta}, time = {max(times)}")
    beta += plus

In [ ]:
#並列ver.
#ある程度持続すれば更新
runge_para["period"]=np.array([0,10000])
competitives = 10

#刻み幅
runge_para["step"] = 0.01
runge_para['start'] = np.load("../laminar_initials/beta0.419_nu0.00017256435676170782_step0.01_2000.0period_laminar.npy")

plus = 1E-3
#nu
runge_para["nu"] = 0.00017256435676170782
beta= 0.418

model = RungeKutta_acceralated.RungeKutta()
lmodel = RungeKutta_acceralated.Long_laminar()

while True:
    if beta > 0.5:
        break
    #betaの値に従いc_n_2,]}}c_n_3を更新
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
    for _ in range(4):
        with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor: 
            task1 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            task2 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            task3 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            task4 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            # task5 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            # task6 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            # task7 = executor.submit(func_for_paralell.competition, runge_para, competitives)
            
            times1, starts1, nus1 = task1.result()
            times2, starts2, nus2 = task2.result()
            times3, starts3, nus3 = task3.result()
            times4, starts4, nus4 = task4.result()
            # times5, starts5, nus5 = task5.result()
            # times6, starts6, nus6 = task6.result()
            # times7, starts7, nus7 = task7.result()
        times = np.concatenate([times1, times2, times3, times4])
        starts = np.concatenate((starts1, starts2, starts3, starts4), axis=1)
        nus = np.concatenate([nus1, nus2, nus3, nus4])

        # times = np.concatenate([times1, times2, times3, times4, times5, times6, times7])
        # starts = np.concatenate((starts1, starts2, starts3, starts4, starts5, starts6, starts7), axis=1)
        # nus = np.concatenate([nus1, nus2, nus3, nus4, nus5, nus6, nus7])
        runge_para['nu'] = nus[np.argmax(times)]
        runge_para['start'] = starts[:, np.argmax(times):np.argmax(times)+1]
        print(max(times))
        if max(times) > runge_para["period"][1]*0.99:
            break
    if max(times) > 600:
        np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{int(max(times))}period_laminar", runge_para["start"])
    print(f"beta={beta}, nu = {runge_para['nu']} time = {max(times)}")
    model.get_arr_latter(**runge_para,latter = 1)
    plt.clf()
    plt.figure(figsize=(40,5))
    plt.plot(abs(model.arr_latter[-1,:]),abs(model.arr_latter[0,:]))
    plt.show()
    beta += plus

In [ ]:
np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{max(times)}period_laminar", runge_para["start"])

In [ ]:
#軌道確認
skip = 1

runge_para['period']=np.array([0,10000])
model.get_arr_latter(**runge_para,latter=1)
Nth_shell = 3
plt.figure(figsize=(20,30))
for i in range(Dimention):
    plt.subplot(Dimention+1,1,i+1)
    plt.ylabel(f"u{i+1}")
    #plt.yscale=("log")
    plt.plot(np.real(model.arr_latter[-1,::skip]),abs(model.arr_latter[i,::skip]),lw=1)

plt.subplot(Dimention+1,1,Dimention+1)
plt.ylabel(f"flux of {Nth_shell}th shell")
plt.plot(np.real(model.arr_latter[-1,:]), lmodel.flux(model.arr_latter[:-1,:], Nth_shell, beta)[0,:])

## stagger and step(自動)

In [ ]:
runge_para["period"]=np.array([0,1000])

#刻み幅
runge_para["step"] = 0.01

runge_para["check"] = 1000
runge_para["progress"] = 1000

runge_para['start'] = np.load("../laminar_initials/beta0.4254000000000001_nu0.0001737395209159338_step0.01_1000chec_10pro_10period.npz")["start"]

plus = 1E-5

#nu
runge_para["nu"] = 0.0001737395209159338
original_nu = runge_para["nu"]*1
#betaの値に従いc_n_2,c_n_3を更新
beta = 0.4254

lmodel = Long_laminar()


#betaを0.5に向けて更新

#ここから
while True:        
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
    

    lmodel.get_laminar(**runge_para)
    #上手くいかなかった時
    if abs(lmodel.arr[0,-1]) == 0:
        #nuを更新
        a=-5
        b=-14
        runge_para["nu"] = original_nu + (10) ** ((b - a) * np.random.rand() + a) * (-1) ** np.random.randint(1,3) #nuを適当に更新(一応負の方向に行きやすい)
        continue
#ここまでを並列化したい。

    original_nu = runge_para["nu"]*1
    runge_para["start"] = lmodel.arr[:-1,0:1]
    np.savez_compressed(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{runge_para['check']}chec_{runge_para['progress']}pro_{runge_para['period'][1]}period", start=lmodel.arr[:-1,0:1], laminar=lmodel.arr, perturbated_time=lmodel.perturbated_time)
    print(f"\n 現在beta={beta},nu={runge_para['nu']}")
    beta += plus

## stagger and step(手動)

In [ ]:
%%time
runge_para["period"]=np.array([0,5000])

#刻み幅
runge_para["step"] = 0.01
skip= 10

check = 1000
progress = 100

#runge_para['start'] = np.load("14次元初期値.npy")

plus = 5E-4

#nu
runge_para["nu"] = 0.00017
#betaの値に従いc_n_2,c_n_3を更新
beta = 0.4174

lmodel = Long_laminar()

#betaを0.5に向けて更新
for i in tqdm(range(1)):
    #c_n_2
    runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
    runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
    #c_n_3
    runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)
    

    lmodel.get_laminar(**runge_para, check = check, progress = progress)
    #上手くいかなかった時
    if abs(lmodel.arr[0,-1]) == 0:
        plt.figure(figsize=(30,40))
        for i in range(len(lmodel.arr[:-1,0])):
            plt.subplot(len(lmodel.arr[:-1,0]),1,i+1)
            plt.ylabel(f"u{i+1}")
            #plt.yscale=("log")
            plt.plot(np.real(zenkai[-1,:]),abs(zenkai[i,:]),lw=0.5)
            for j in range(len(lmodel.perturbated_time)):
                plt.vlines(x=lmodel.perturbated_time[j], ymin=np.min(abs(zenkai[i,:])), ymax=np.max(abs(zenkai[i,:])),colors="red")
        print(f'最後のbeta={beta - plus}')
        break
        
    runge_para["start"] = lmodel.arr[:-1,-1:]
    zenkai = lmodel.arr
    np.save(f"../laminar_initials/beta{beta}_nu{runge_para['nu']}_step{runge_para['step']}_{check}check_{progress}pro",lmodel.arr[:-1,0:1])
    print(f"\r 現在beta={beta}", end="")
    beta += plus

## 初期値の調整

In [ ]:
runge_para['start'] = np.load("../laminar_initials/beta0.4215000000000002_nu0.00017072648529039704_step0.01_1000chec_10pro_10period.npz")["start"]

In [ ]:
e = int(len(zenkai[0,:])//100*50)
runge_para['start'] = zenkai[:-1,e:e+1]

In [ ]:
d = 6
runge_para['start'] = model.arr_latter[:-1,int(len(model.arr_latter[0,:])//100*d):int(len(model.arr_latter[0,:])//100*d)+1]

In [ ]:
#極大値極小値の範囲を知りたい
def loc_max_7(arr):
    N = 0
    z_list = np.zeros((len(arr[:,0]),1))
    for j in range(len(arr[0,:]) - 10):
        #7点とって極大判定
        if (np.abs(arr[N,j+1]) - np.abs(arr[N,j]) > 0
        and np.abs(arr[N,j+2]) - np.abs(arr[N,j+1]) > 0
        and np.abs(arr[N,j+3]) - np.abs(arr[N,j+2]) > 0
        and np.abs(arr[N,j+4]) - np.abs(arr[N,j+3]) < 0
        and np.abs(arr[N,j+5]) - np.abs(arr[N,j+4]) < 0
        and np.abs(arr[N,j+6]) - np.abs(arr[N,j+5]) < 0):
            z_list = np.append(z_list,np.abs(arr[:,j+3:j+4]),axis=1)
    return z_list[:,1:]

#c_N+1が極小値を取る時の全変数(+時間)をndarrayで返す関数

def loc_min_7(arr):
    N = 0
    z_list = np.zeros((len(arr[:,0]),1))
    for j in range(len(arr[0,:]) - 10):
        #7点とって極大判定
        if (np.abs(arr[N,j+1]) - np.abs(arr[N,j]) < 0
        and np.abs(arr[N,j+2]) - np.abs(arr[N,j+1]) < 0
        and np.abs(arr[N,j+3]) - np.abs(arr[N,j+2]) < 0
        and np.abs(arr[N,j+4]) - np.abs(arr[N,j+3]) > 0
        and np.abs(arr[N,j+5]) - np.abs(arr[N,j+4]) > 0
        and np.abs(arr[N,j+6]) - np.abs(arr[N,j+5]) > 0):
            z_list = np.append(z_list,np.abs(arr[:,j+3:j+4]),axis=1)
    return z_list[:,1:]

min=loc_min_7(model.arr_latter)
max=loc_max_7(model.arr_latter)
print(np.max(min[0,:]),np.min(min[0,:]))
print(np.max(max[0,:]),np.min(max[0,:]))

In [ ]:
a, b = 3, 4
fig = plt.figure()
ax = plt.subplot(1,1,1)
ax.set_xlabel(f"u{a+1}")
ax.set_ylabel(f'u{b+1}')
ax.set_xscale("log")
ax.set_yscale("log")
ax.plot(np.abs(model.arr_latter[a,:50000]),np.abs(model.arr_latter[b,:50000]),lw=0.1)

In [ ]:
#lmodel.perturbated_time

In [ ]:
skip=1
plt.figure(figsize=(30,40))
arr=np.load("../laminar_initials/beta0.426_nu0.00017476988409663425_step0.01_1317.4799999989468period_laminar.npz")["laminar"]
#perturbated_time=np.load("../laminar_initials/beta0.42525_nu0.00017380418305597015_step0.01_1500chec_10pro_100period.npz")["perturbated_time"]


for i in range(len(arr[:-1,0])):
    plt.subplot(len(arr[:-1,0]),1,i+1)
    plt.ylabel(f"u{i+1}")
    #plt.yscale=("log")
    plt.plot(np.real(arr[-1,::skip]),abs(arr[i,::skip]),lw=0.5)
    # for j in range(len(perturbated_time)):
    #     plt.vlines(x=perturbated_time[j], ymin=np.min(abs(arr[i,:])), ymax=np.max(abs(arr[i,:])),colors="red")
#plt.savefig(fname=f"../laminar/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_{check}check_{progress}progress.png")